In [4]:
import mlflow
import pandas as pd

# Set the tracking URI to the MLFLow Server
mlflow.set_tracking_uri('http://127.0.0.1:5000')

# Retrieve experiments
experiments = mlflow.search_experiments()

# Display the available experiments
experiments

# Display the available experiments
experiments = mlflow.search_experiments()

#Display the available experiments
for exp in experiments:
    print(f"Experiment ID: {exp.experiment_id}, Name: {exp.name}")

# Choose an experiment
experiment_id = experiments[0].experiment_id

# Retrieve runs from selected experiment
runs = mlflow.search_runs(experiment_ids=[experiment_id])

# Convert runs to a DataFrame for easier manipulation
runs_df = pd.DataFrame(runs)

# Display the first few rows of the DAtaFrame
runs_df.head()

Experiment ID: 0, Name: Default


,run_id,experiment_id,status,artifact_uri,start_time,end_time
